In [3]:
import tensorflow as tf
from tensorflow.python import keras
#import tensorflow_graphics as tfg
import tensorflow_addons as tfa
import tensorflow_probability as tfp
from image_interpolation import ImageInterpolator

2022-09-26 17:49:01.812996: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-26 17:49:02.031509: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-09-26 17:49:02.895874: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-09-26 17:49:02.895931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [4]:
image=tf.constant([[[[1.0],[2.0],[3.0]],[[4.0],[5.0],[6.0]],[[7.0],[8.0],[9.0]]]])
queries=tf.Variable([[[1.1,1.3]]])

2022-09-26 17:49:06.513179: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-09-26 17:49:06.513230: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ideapad5pro): /proc/driver/nvidia/version does not exist
2022-09-26 17:49:06.514883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
with tf.GradientTape() as tape:
    z=tfa.image.interpolate_bilinear(image,queries)
print(tape.gradient(z,queries))

tf.Tensor([[[3. 1.]]], shape=(1, 1, 2), dtype=float32)


In [13]:
with tf.GradientTape() as tape:
    z=tfp.math.batch_interp_regular_nd_grid(queries,[0.0,0.0],[3.0,3.0],image,axis=-3)
    print(z)
print(tape.gradient(z,queries))

tf.Tensor([[[4.0666666]]], shape=(1, 1, 1), dtype=float32)
tf.Tensor([[[2.0000002 0.6666665]]], shape=(1, 1, 2), dtype=float32)


In [8]:
interpolator=ImageInterpolator(grid_dim=(3,3))
interpolator(image,tf.constant([[0.0,0.0,1.0]]))

<tf.Tensor: shape=(1, 3, 3, 1), dtype=float32, numpy=
array([[[[1.],
         [2.],
         [3.]],

        [[4.],
         [5.],
         [6.]],

        [[7.],
         [8.],
         [9.]]]], dtype=float32)>

In [20]:
x=tf.Variable([[0.0,0.0,1.0]])
with tf.GradientTape() as tape:
    interpolation=interpolator(image,x)
    s=tf.reduce_sum(interpolation,axis=1)
    s2=tf.reduce_sum(s,axis=1)
    print(s)
    print(s2)
print(tape.gradient(s2,x))

tf.Tensor(
[[[12.]
  [15.]
  [18.]]], shape=(1, 3, 1), dtype=float32)
tf.Tensor([[45.]], shape=(1, 1), dtype=float32)
tf.Tensor([[54. 18. 36.]], shape=(1, 3), dtype=float32)
